# Create an API Function


In [1]:
from requests import post as rpost

def call_llama(prompt):
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": "llama3.1",
        "prompt": prompt,
        "stream": False,
    }

    response = rpost(
        "http://localhost:11434/api/generate",
        headers=headers,
        json=payload
    )
    return response.json()["response"]

# or use ollama python library 

# Say hello to the LLM

In [2]:
call_llama(prompt="hello")

'Hello! How are you today? Is there something I can help you with or would you like to chat?'

# Create a LangChain LLM


In [3]:
from langchain_core.language_models.llms import LLM

class LLaMa(LLM):
    def _call(self, prompt, **kwargs):
        return call_llama(prompt)

    @property
    def _llm_type(self):
        return "llama-3.1-8b"

# Docs

In [4]:
# could be loaded from pdfs or company website 
documents = [
    {"content": "What is your return policy? Our return policy allows customers to return items within 30 days of purchase for a full refund."},
    {"content": "How long does shipping take? Standard shipping typically takes 5-7 business days, while expedited shipping takes 2-3 business days."},
    {"content": "Do you offer international shipping? Yes, we offer international shipping to select countries. Please check our website for more details."},
    {"content": "What payment methods do you accept? We accept all major credit cards, PayPal, and Apple Pay."},
    {"content": "How can I track my order? You can track your order using the tracking link provided in your confirmation email."},
    {"content": "What should I do if I receive a damaged item? If you receive a damaged item, please contact our customer service within 48 hours for assistance."},
    {"content": "Can I change or cancel my order? You can change or cancel your order within 1 hour of placing it. After that, it may be processed for shipping."},
    {"content": "Do you have a loyalty program? Yes, we have a loyalty program that rewards customers with points for every purchase, which can be redeemed for discounts."},
    {"content": "What is your privacy policy? Our privacy policy outlines how we collect, use, and protect your personal information."},
    {"content": "How can I contact customer support? You can contact customer support via email at support@example.com or by calling our hotline at 1-800-555-0199."},
]


# Embeddings

In [5]:
from tqdm.autonotebook import tqdm, trange # to show progress 
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings



texts = [doc["content"] for doc in documents]

retriever = FAISS.from_texts(
    texts,
    HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
).as_retriever(k=5)

C:\Users\Motaz PC\AppData\Local\Temp\ipykernel_42228\1939657843.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange # to show progress


# Create the Prompt Template

In [6]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

faq_template = """
You are a chat agent for my E-Commerce Company. As a chat agent, it is your duty to help the human with their inquiry and make them a happy customer.

Help them, using the following context:
<context>
{context}
</context>
"""

faq_prompt = ChatPromptTemplate.from_messages([
    ("system", faq_template),
    MessagesPlaceholder("messages")
])

# Create Document and Retriever Chains


In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

document_chain = create_stuff_documents_chain(LLaMa(), faq_prompt)

def parse_retriever_input(params):
    return params["messages"][-1].content

retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever
).assign(answer=document_chain)

# Prompt Your RAG Agent


In [8]:
from langchain.schema import HumanMessage

response = retrieval_chain.invoke({
    "messages": [
        HumanMessage("I received a damaged item. I want my money back.")
    ]
})

print(response)

{'messages': [HumanMessage(content='I received a damaged item. I want my money back.', additional_kwargs={}, response_metadata={})], 'context': [Document(metadata={}, page_content='What should I do if I receive a damaged item? If you receive a damaged item, please contact our customer service within 48 hours for assistance.'), Document(metadata={}, page_content='What is your return policy? Our return policy allows customers to return items within 30 days of purchase for a full refund.'), Document(metadata={}, page_content='How can I contact customer support? You can contact customer support via email at support@example.com or by calling our hotline at 1-800-555-0199.'), Document(metadata={}, page_content='Can I change or cancel my order? You can change or cancel your order within 1 hour of placing it. After that, it may be processed for shipping.')], 'answer': "I'm so sorry to hear that you received a damaged item.\n\nDon't worry, we're here to help! According to our policies, if you r

In [11]:
response.keys()

dict_keys(['messages', 'context', 'answer'])

In [9]:
response['messages']

[HumanMessage(content='I received a damaged item. I want my money back.', additional_kwargs={}, response_metadata={})]

In [10]:
response['context']

[Document(metadata={}, page_content='What should I do if I receive a damaged item? If you receive a damaged item, please contact our customer service within 48 hours for assistance.'),
 Document(metadata={}, page_content='What is your return policy? Our return policy allows customers to return items within 30 days of purchase for a full refund.'),
 Document(metadata={}, page_content='How can I contact customer support? You can contact customer support via email at support@example.com or by calling our hotline at 1-800-555-0199.'),
 Document(metadata={}, page_content='Can I change or cancel my order? You can change or cancel your order within 1 hour of placing it. After that, it may be processed for shipping.')]

In [13]:
print(response['answer'])

I'm so sorry to hear that you received a damaged item.

Don't worry, we're here to help! According to our policies, if you receive a damaged item, please contact our customer service within 48 hours for assistance. You can reach us via email at support@example.com or by calling our hotline at 1-800-555-0199.

Can I provide you with the details on how to proceed? We'll be happy to assist you in getting a full refund or replacing the damaged item, whichever you prefer!
